# Landsat LST Example

In this notebook, we will demonstrate how to fetch and visualize Landsat Land Surface Temperature (LST) data using the `ee_lst` library and Google Earth Engine API. We'll use `geemap` for interactive map visualizations with layer controls.
The layers are derived from the `example_1.py` script, and the explanations are based on the paper titled ["A Google Earth Engine Free App for Estimating Near-Real Time Daily Surface Temperature from Landsat Data"](https://www.mdpi.com/2072-4292/12/9/1471/pdf?version=1589365556).

In [1]:
import ee

In [2]:
# Initialize the Earth Engine API
if not ee.data._initialized:
    try:
        ee.Initialize()
    except Exception as e:
        print("Please authenticate Google Earth Engine first.")
        ee.Authenticate()
        ee.Initialize()

## Fetching Landsat Data

We will fetch Landsat data for a specific region and time range. The data will include added variables such as NDVI, FVC, TPW, EM, and LST.

In [ ]:
%pip install -q git+https://github.com/lunasilvestre/ee_lst.git

In [3]:
from modules.landsat_lst import fetch_landsat_collection

In [4]:
# Define parameters
geometry = ee.Geometry.Rectangle([-8.91, 40.0, -8.3, 40.4])
satellite = 'L8'
date_start = '2023-05-15'
date_end = '2023-10-15'
use_ndvi = True

# Get Landsat collection with added variables: NDVI, FVC, TPW, EM, LST
landsat_coll = fetch_landsat_collection(satellite, date_start, date_end, geometry, use_ndvi)
landsat_coll = landsat_coll.sort('TIMESTAMP')

## Visualizing Landsat Data with geemap

Now that we have fetched the Landsat data, let's visualize it on an interactive map using `geemap`. We will display the Land Surface Temperature (LST) layer.

In [13]:
%pip install ipyleaflet
# !jupyter-nbextension enable --py --sys-prefix ipyleaflet

/bin/bash: line 1: jupyter-nbextension: command not found


In [18]:
!jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-leaflet

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

optional arguments:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot

Jupyter command `jupyter-labextension` not found.


In [19]:
from ipywidgets import IntSlider
slider = IntSlider()
slider

IntSlider(value=0)

In [17]:
import ipyleaflet
m = ipyleaflet.Map()
m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [14]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [15]:
# import ee
import geemap

In [ ]:
# geemap.update_package()

## Create an interactive map
The default basemap is Google Maps. Additional basemaps can be added using the Map.add_basemap() function.

In [16]:
Map = geemap.Map() #center=[40,-100], zoom=4)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Visualizing Landsat Data with folium

Now that we have fetched the Landsat data, let's visualize it on an interactive map using `folium`. We will display the Land Surface Temperature (LST) layer.

In [ ]:
import folium
from folium import plugins

In [ ]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        tile_layer = None
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            tile_layer = folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            )
            tile_layer.add_to(self)

        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            tile_layer = folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            )
            tile_layer.add_to(self)

        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            tile_layer = folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        )
        tile_layer.add_to(self)

        print(tile_layer)
        return tile_layer
    except:
        print("Could not display {}".format(name))
        return None


# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Get Landsat collection first image
# ex_image = landsat_coll.first()

# Visualization parameters
cmap1 = ['blue', 'cyan', 'green', 'yellow', 'red']
cmap2 = ['F2F2F2','EFC2B3','ECB176','E9BD3A','E6E600','63C600','00A600']


# Add the Earth Engine layers to the folium map
layers = [
    {'band': 'TPW', 'vis_params': {'min': 0, 'max': 60, 'palette': cmap1}, 'name': 'TCWV'},
    {'band': 'TPWpos', 'vis_params': {'min': 0, 'max': 9, 'palette': cmap1}, 'name': 'TCWVpos'},
    {'band': 'FVC', 'vis_params': {'min': 0, 'max': 1, 'palette': cmap2}, 'name': 'FVC'},
    {'band': 'EM', 'vis_params': {'min': 0.9, 'max': 1.0, 'palette': cmap1}, 'name': 'Emissivity'},
    {'band': 'B10', 'vis_params': {'min': 290, 'max': 320, 'palette': cmap1}, 'name': 'TIR BT'},
    {'band': 'LST', 'vis_params': {'min': 290, 'max': 320, 'palette': cmap1}, 'name': 'LST'} #,
    # {'band': 'RGB', 'vis_params': {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3}, 'name': 'RGB'}
]

# https://stackoverflow.com/questions/64792041/white-gap-between-python-folium-map-and-jupyter-notebook-cell
# Create a folium map object
# from branca.element import Figure
# fig = Figure(height=500) #width=600, 
my_map = folium.Map(location=[40.2, -8.6], zoom_start=10, height=500)
# fig.add_child(my_map)

tile_layers = [] 
for layer in layers:
    # tile_layer = my_map.add_ee_layer(ex_image.select(layer['band']), layer['vis_params'], layer['name'])
    tile_layer = my_map.add_ee_layer(landsat_coll.select(layer['band']), layer['vis_params'], layer['name'])
    if tile_layer:
        tile_layers.append(tile_layer)    

# Add WmsTileLayers to time control.
time = plugins.TimestampedWmsTileLayers(tile_layers)
time.add_to(my_map)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map
my_map

## Explanation of Layers

### TCWV (Total Column Water Vapor)
This layer represents the Total Column Water Vapor (TCWV). Water vapor plays a crucial role in the Earth's energy balance, and its distribution is essential for weather forecasting and climate modeling. In the context of urban areas, understanding TCWV can provide insights into urban heat island effects and local climate variations.

### TCWVpos
This layer is a positive representation of the TCWV. It emphasizes areas with higher water vapor concentrations, which can be particularly relevant in urban climate studies to identify areas of increased humidity or potential heat stress.

### FVC (Fractional Vegetation Cover)
The Fractional Vegetation Cover (FVC) layer provides a measure of the proportion of ground covered by vegetation. In urban areas, vegetation can play a significant role in mitigating the urban heat island effect, providing shade, and improving air quality. Thus, understanding FVC can help in urban planning and sustainability efforts.

### Emissivity
Emissivity represents the efficiency of a surface in emitting thermal radiation. Different surfaces, such as buildings, roads, and vegetation, have different emissivity values. In urban areas, understanding surface emissivity can help in assessing the thermal properties of various surfaces and their contribution to local temperature variations.

### TIR BT (Thermal Infrared Brightness Temperature)
This layer represents the brightness temperature in the thermal infrared spectrum. It provides a direct measure of the surface temperature, which is crucial for understanding urban heat islands and local temperature variations.

### LST (Land Surface Temperature)
The Land Surface Temperature (LST) layer provides a measure of the temperature of the Earth's surface. In urban areas, LST can vary significantly due to factors like urban heat islands, surface materials, and vegetation cover. Understanding LST is essential for urban climate studies and health assessments.

### RGB
This layer provides a true-color representation of the area using the Red, Green, and Blue bands. It gives a visual context to the other layers and helps in understanding the spatial distribution of features like vegetation, water bodies, and urban infrastructure.

The methodologies and significance of these layers are discussed in detail in the paper titled "[An Open-Source Tool for Automated Quality Assessment of Landsat Surface Temperature over Urban Areas](https://www.mdpi.com/2072-4292/12/9/1471/pdf?version=1589365556)" by Lorenzo Bigagli, Massimiliano Nole, Simone Mantovani, and Mattia Crespi.